In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def return_dir_files(dir_path: str):
    '''
    Given a directory, return file names contained within
    '''
    fnames = os.listdir(dir_path)

    return fnames

In [ ]:
test=return_dir_files(dir_path='eforms/form_714_2023')
print(test)

['Form_714_2023_City_of_Homestead_Q4_5_21_2024__8_08_AM_170312.xml', 'Form_714_2023_New_York_State_Electric_&_Gas_Corporation_Q4_5_15_2024__7_12_AM_170140.xml', 'Form_714_2023_Basin_Electric_Power_Cooperative_Q4_5_21_2024__4_13_PM_170367.xml', 'Form_714_2023_Southwest_Power_Pool__Inc__Q4_5_14_2024__4_00_PM_170132.xml', 'Form_714_2023_Square_Butte_Electric_Cooperative_Q4_5_23_2024__4_28_PM_170745.xml', 'Form_714_2023_Tampa_Electric_Company_Q4_5_22_2024__6_47_AM_170377.xml', 'Form_714_2023_Southern_Power_Company_Q4_5_29_2024__8_43_AM_181031.xml', 'Form_714_2023_Oklahoma_Municipal_Power_Authority_Q4_4_24_2024__10_35_AM_169794.xml', 'Form_714_2023_Public_Service_Company_of_Colorado_Q4_6_7_2024__5_30_PM_181934.xml', 'Form_714_2023_Salt_River_Project_Agricultural_Improvement_and_Power_District_Q4_5_29_2024__1_25_PM_181090.xml', 'Form_714_2023_City_of_Homestead_Q4_5_17_2024__10_30_AM_170214.xml', 'Form_714_2023_Orlando_Utilities_Commission_Q4_6_3_2024__8_30_AM_181540.xml', 'Form_714_2023_Wolv

In [4]:
def process_xml_data(xml_fpath: str):
    '''
    purpose is to return a processed df when provided with topics to grab
    '''
    df = pd.read_xml(xml_fpath)
    #print(df.info())

    # preset list of data columns we will be grabbing
    topics ={
        "forecast_year": 'PlanningAreaHourlyDemandAndForecastYear',
        "ann_net_load": 'PlanningAreaHourlyDemandAndForecastForecastOfAnnualNetEnergyForLoad',
        "summer_forecast": 'PlanningAreaHourlyDemandAndForecastSummerForecast',
        "winter_forecast": 'PlanningAreaHourlyDemandAndForecastWinterForecast',
        #"respondent_id": "RespondentIdentificationCode",
       # "respondent_name": 'RespondentLegalName',
    }

    # refined topics list
    topics_dict = pd.DataFrame()

    # iterate through list and grab data while converting into float
    for key,value in topics.items():
        temp_df = df[f'{value}'].dropna()

        # this is a list of values we want to convert to integers
        convert_to_ints = ['ann_net_load', 'summer_forecast', 'winter_forecast']
        convert_to_str = ['respondent_name', 'respondent_id']

        if key in convert_to_ints:
            temp_list = list(temp_df.astype(int))

        elif key in convert_to_str:
            temp_list = list(temp_df.astype(str))

        else:
            temp_list = list(pd.to_datetime(temp_df, format="%Y").dt.year)


        if temp_list[-1] <= temp_list[0]:
            sorted_temp_list = sorted(temp_list)
            print(f"had to sort: {xml_fpath}! please check data (=")
            topics_dict[f'{key}'] = sorted_temp_list
        
        else:
            topics_dict[f'{key}'] = temp_list

    # no we are adding the name of the entity name under a new column
    topics_dict['respondent_name'] = list(df['RespondentLegalName'].dropna())[0]

    # also adding report year and period and identification code
    topics_dict['report_year'] = list(df['ReportYear'].dropna())[0]
    topics_dict['report_period'] = list(df['ReportPeriod'].dropna())[0]
    topics_dict['entity_id'] = list(df['CompanyIdentifier'].dropna())[0]
    topics_dict['entity_id'] = topics_dict['entity_id'].astype(str)

    return topics_dict

In [5]:
# function to combine dataframes
def combine_714_dfs(xml_fpaths: list):

    # master df where we will aggregate all xml dfs
    master_df = pd.DataFrame()

    for xml_fpath in xml_fpaths:
        temp_df = process_xml_data(xml_fpath=xml_fpath)
        master_df = pd.concat([master_df, temp_df])

    return master_df

Now let's test whether the functions we created work!

In [7]:
# testing to see what we can pull
x = process_xml_data(xml_fpath='form_714_2023/Form_714_2023_ALABAMA_POWER_COMPANY_Q4_5_28_2024__9_43_AM_180926.xml')
print(x)

/var/folders/lv/_2pd__3j4tl2w9g39c7ybbs00000gn/T/ipykernel_97486/3559392680.py:5: FutureWarning: Passing literal xml to 'read_xml' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_xml(xml_fpath)


ImportError: lxml not found, please install or use the etree parser.